In [18]:
import pandas as pd

In [19]:
df = pd.read_csv("spotify_millsongdata.csv")

In [20]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [21]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [22]:
df.shape

(57650, 4)

In [23]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [24]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [25]:
df.head(10)

,artist,song,text
0,Jim Croce,Dreamin' Again,Don't you know I had a dream last night \r\nT...
1,Johnny Cash,I Heard That Lonesome Whistle Blow,I was riding number nine \r\nHeading South fr...
2,Kris Kristofferson,Shake Hands With The Devil,I know some people say that I'm the devil in d...
3,Toto,Melanie,"Watching, driving through the night, I'm all a..."
4,Miley Cyrus,Liberty Walk,Don't live a lie \r\nThis is your one life \...
5,Ziggy Marley,Melancholy Mood,A little collie weed to ease I grief \r\nToo ...
6,Drake,Fear,This is why I do this shit. \r\nI think they ...
7,Steely Dan,Rikki Don't Lose That Number,"We hear you're leaving, that's OK \r\nI thoug..."
8,Offspring,The Noose,Well our souls are all mistaken in the same mi...
9,Roxette,Star,"You keep on walkin' on the other side, \r\nOt..."


In [26]:
df['text'][0]

"Don't you know I had a dream last night  \r\nThat you were here with me  \r\nLyin' by my side so soft and warm  \r\nAnd we talked a while  \r\nAnd shared a smile  \r\nAnd then we shared the dawn  \r\nBut when I woke up  \r\nOh, my dream it was gone  \r\nDon't you know I had a dream last night  \r\nYou were here with me  \r\nLyin' by my side so soft and warm  \r\nAnd you said you'd thought it over  \r\nSaid that you were comin' home  \r\nBut then I woke up  \r\nFrom my dream it was gone  \r\nI'm not the same  \r\nCan you blame me  \r\nIs it hard to understand  \r\nI can't forget  \r\nYou can't change me  \r\nI am not that kind of man  \r\nDon't you know I had a dream last night  \r\nWhen everything was still  \r\nAnd you were by my side so soft and warm  \r\nAnd I dreamed that we were lovers  \r\nIn the lemon-scented rain  \r\nBut the I woke up  \r\nOh, I found that again  \r\nI had been dreamin'  \r\nDreamin' again  \r\nI had been dreamin'  \r\nDreamin' again\r\n\r\n"

In [27]:
# df = df.sample(5000)

In [28]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [29]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [30]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [31]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [33]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [34]:
similarity[0]

array([1.        , 0.01277827, 0.01244836, ..., 0.01525594, 0.0643679 ,
       0.0283941 ])

In [35]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [36]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [37]:
recommendation('Crying Over You')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))